In [ ]:
COUNT_DIGITS = {
    1: "a haon",
    2: "a dó",
    3: "a trí",
    4: "a ceathair",
    5: "a cúig",
    6: "a sé",
    7: "a seacht",
    8: "a hocht",
    9: "a naoi"
}

In [ ]:
DIALECT_MU = {
    "anseo": "anso",
    "ansin": "ansan",
    "dtús", "dtúis",
    "tosú": "tosnú",
    "tosóidh": "tosnóidh",
    "atá": "athá",
    "atá tú": "athánn tú",
    "léi": "léithe",
    "arís": "aríst"
}

In [ ]:
PAGE_SPECIFIC = {
    "181": {
        "Mar 'Tháinig": "Mar a Tháinig",
        "Cabidil": "Caibidil",
        "mar Chuir": "mar a Chuir"
    },
    "190": {
        "nuair d'fhéach": "nuair a d'fhéach"
    },
    "191": {
        "le mo": "lem"
    }
}

In [ ]:
def skip_if_colon(text):
    if ":" in text:
        return ""
    else:
      return text

def skip_art(text):
    if text.startswith("(Ealaín:"):
        return ""
    else:
        return text

In [11]:
import requests
from bs4 import BeautifulSoup

def get_page_text_and_audio(url, poetry=True):
    req = requests.get(url)
    if req.status_code != 200:
        return None
    soup = BeautifulSoup(req.text, 'html.parser')

    page_text = soup.find("div", {"class": "page-text"})

    audio_file = ""
    audio = page_text.find("audio")
    if audio is not None:
        source = audio.find("source")
        if source is not None:
            audio_file = "https://www.leighleat.com" + source["src"]
    audio.decompose()

    if poetry:
        out_text = page_text.text.strip()
    else:
        pass

    return out_text, audio_file

In [12]:
get_page_text_and_audio("https://www.leighleat.com/poems/26")

('Damhán Alla\nDamhán alla\nDamhán alla\nAr an mballa\nAr an mballa\nTháinig éan\nTháinig éan\nÓ mo léan\nÓ mo léan!',
 'https://www.leighleat.com/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaHBBdmdNIiwiZXhwIjpudWxsLCJwdXIiOiJibG9iX2lkIn19--1e2441aa5cfdfdc2ed88fafc4a1ed354739f6af6/damhan%20alla.mp3')